In [1]:
# !pip install pandas	
# !pip install numpy
# !pip install tqdm	
# !pip install matplotlib

In [911]:
# 라이브러리
%load_ext autoreload
%autoreload 2
    
import random
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from numpy.random import randint, rand

import ga_runner


SEED_VALUE = 20250526
DIVIDE_VALUE = 123
VERBOSE_VALUE = False
MACHINE_INFO_FILE = "./data/machine_info.csv"
ORDER_INFO_FILE = "./data/TalkFile_final_orders_completed_ko.csv"
# ORDER_INFO_FILE = "./data/order_info.csv"
ORDER_INFO_NL_FILE = "./data/order_info_nl.csv"
ORDER_INFO_NL_ENG_FILE = "./data/order_info_nl_eng.csv"
GT_N5_PKL_FILE = 'gt_n5_augmented_20250613_divide_date.pkl'
# GT_N5_PKL_FILE = 'gt_n5_20250613_divide_123.pkl'
ORIGIN_GT_N5_FILE = "./dataset_n5_20250526.csv"
REFACTORY_GT_N5_FILE = "./augmented_dataset_n5_20250613_divide_date.csv"
# REFACTORY_GT_N5_FILE = "./gt_n5__20250613_divide_123.csv"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [912]:
# 데이터 불러오기
machine_info = pd.read_csv(MACHINE_INFO_FILE)
order_info = pd.read_csv(ORDER_INFO_FILE)
order_info_nl = pd.read_csv(ORDER_INFO_NL_FILE, header=None)
order_info_nl_eng = pd.read_csv(ORDER_INFO_NL_ENG_FILE)

In [913]:
order_info_nl.columns = ['영업납기', '중산도면', '단가', '수량', '선급', '자연어주문']

In [914]:
# 전처리 (1): column name 변경
order_info = order_info.rename(columns={
    '영업납기': 'time',
    '중산도면': 'item',
    '단가': 'cost',
    '수량': 'qty',
    '선급': 'urgent'
})

machine_info = machine_info.rename(columns={
    'JSDWG': 'item',
    'MCNO': 'machine',
    'AVG_CT': 'capacity'
})

order_info_nl = order_info_nl.rename(columns={
    '영업납기': 'time',
    '중산도면': 'item',
    '단가': 'cost',
    '수량': 'qty',
    '선급': 'urgent',
    '자연어주문': 'nl_order'
})

order_info_nl_eng.columns = ['nl_order']

In [915]:
# 머신인포 dropna, 4자리 숫자 문자열로 변환
machine_info.dropna(inplace=True)
machine_info['machine'] = machine_info['machine'].apply(lambda x: str(int(x)).zfill(4))

In [916]:
# 아이템 -> item01
# qty -> 그대로
# 날짜 -> 일수(날짜 - 2021-01-01) date001(1~365) / 자연어는 수정X
# 선급 -> 일단 자연어 그대로
# cost -> 일단 그대로
# machine -> machine01
# capacity -> 일단 그대로

In [917]:
# 아이템 -> item01
unique_items = sorted(order_info['item'].unique(), reverse=True)

items_mapping = {
        item: f"item{str(i+1).zfill(2)}"
        for i, item in enumerate(unique_items)
    }

def change_item_name(target):
    result = str(target)
    for k, v in items_mapping.items():
        result = result.replace(k, v)

    return result

change_item_name('57791')

'57791'

In [918]:
machine_info['item'] = machine_info['item'].apply(change_item_name)
cond = machine_info['item'].isin(items_mapping.values())
machine_info[cond].head()

,item,machine,capacity
0,item6617,0433,1.08
1,item6616,0404,7.13
2,item6616,0408,4.67
3,item6616,0410,4.50
4,item6616,0416,3.92


In [919]:
order_info['item'] = order_info['item'].apply(change_item_name)
order_info.head()
len(order_info)

19851

In [920]:
order_info_nl['item'] = order_info_nl['item'].apply(change_item_name)
order_info_nl['nl_order'] = order_info_nl['nl_order'].apply(change_item_name)
order_info_nl.head()
len(order_info_nl)

127

In [921]:
order_info_nl_eng['nl_order'] = order_info_nl_eng['nl_order'].apply(change_item_name)
order_info_nl_eng.head()

,nl_order
0,"item727, right? 318? You can do it by May 13th..."
1,"Oh, I just got the item728 drawing, and it say..."
2,"There, I need to do 19 drawings for item6402. ..."
3,"Hey, look quickly. item5038 4 I have to do it ..."
4,"item4760, right? Doing 196? You can do it unti..."


In [922]:
# 날짜 -> 일수(날짜 - 2021-01-01) (1~365)
def change_time_name(target):
    time = pd.to_datetime(target)
    base_date = pd.to_datetime('2021-01-01')

    return f'date{str((time-base_date).days+1).zfill(3)}'

change_time_name('2021-01-01'), change_time_name('2021-12-31'),  change_time_name('2021-03-13')

('date001', 'date365', 'date072')

In [923]:
order_info['time'] = order_info['time'].apply(change_time_name)
order_info.head()

,item,cost,qty,time,urgent
0,item6617,6498,1,date077,1
1,item6617,6498,1,date023,0
2,item6617,6498,2,date031,0
3,item6616,15483,4,date266,0
4,item6616,15483,30,date074,1


In [924]:
order_info_nl['time'] = order_info_nl['time'].apply(change_time_name)
order_info_nl.head()

,time,item,cost,qty,urgent,nl_order
0,date133,item727,25870,318,검사품,item727 맞지? 318개 하는거? 21년 5월 13일까지 하면 되구 단가가 2...
1,date144,item728,16229,383,검사품,헐 방금 item728 도면 왔는데 21년 5월 24일까지래. 383개 해야되구 1...
2,date150,item6402,8333,19,NaN,"저기 item6402 도면 19개 해야돼. 21년 5월 30일까지구 8,333원. ..."
3,date154,item5038,36533,4,NaN,어우 빨리 봐봐. item5038 4개 21년 6월 3일까지 해야되는데 어떡하지? ...
4,date169,item4760,45500,196,검사품,item4760 맞지? 196개 하는거? 21년 6월 18일까지 하면 되구 단가가 ...


In [925]:
# machine -> machine01
unique_machines = sorted(machine_info['machine'].dropna().unique())

machines_mapping = {
        machine: f"machine{str(i+1).zfill(2)}"
        for i, machine in enumerate(unique_machines)
    }

def change_machine_name(target):
    result = str(target)
    for k, v in machines_mapping.items():
        result = result.replace(k, v)

    return result

change_machine_name('0421')

'machine19'

In [926]:
machine_info['machine'] = machine_info['machine'].apply(change_machine_name)
machine_info.head()

,item,machine,capacity
0,item6617,machine28,1.08
1,item6616,machine04,7.13
2,item6616,machine08,4.67
3,item6616,machine10,4.50
4,item6616,machine14,3.92


In [927]:
# 전처리 - order
def preprocess_order(df):
    # 긴급 생산 요건이 있는 경우 1, 그렇지 않은 경우는 0
    df['urgent'] = df['urgent'].fillna(0)

    for i in range(len(df)):
        if df.loc[i, 'urgent'] != 0:
            df.loc[i, 'urgent'] = 1

    df.dropna(axis=0, inplace=True)

    # 같은 종류의 order 묶기
    df = df.groupby(['time','item','cost','urgent']).sum().reset_index()
    # df = df.groupby(['time','item','cost','urgent'], as_index=False).first() 
    
    return df

# 전처리 - machine
def preprocess_machine(df):
    df.dropna(axis=0, inplace=True)

    return df

In [928]:
def make_dataset(sampled_orders, machine_info):
    dataset_for_ga = []
    dataset_for_nl = []
    dataset_for_order = []

    for data in sampled_orders:
        data = data.reset_index(drop=True)  # 여기서 인덱스 재설정
        order = data.iloc[:, :5]
        nl = data.iloc[:, 5:]

        order = preprocess_order(order)
        dataset_for_order.append(order)

        order = pd.merge(order, machine_info, on='item', how='inner')
        dataset_for_ga.append(order)
        dataset_for_nl.append(nl)

    return dataset_for_ga, dataset_for_nl, dataset_for_order


# raw_order_info_nl_eng = order_info_nl_eng.copy()
# 1. 정형 + 자연어 정보 합치기
order_info_nl_eng = pd.concat([order_info, order_info_nl_eng], axis=1)

# 2. 'time' 기준으로 그룹화하여 각 날짜에 해당하는 주문을 하나의 샘플로 저장
sampled_orders = [group.reset_index(drop=True)
                  for _, group in order_info_nl_eng.groupby('time')]

# 3. 'date1', 'date2', ... 순서대로 정렬 (문자열이지만 숫자 기준 정렬)
sampled_orders.sort(key=lambda df: int(df['time'].iloc[0].replace('date', '')))

# 4. 머신 정보 전처리
machine_info = preprocess_machine(machine_info)

# 5. 데이터셋 생성
dataset_for_ga, dataset_for_nl, dataset_for_order = make_dataset(sampled_orders, machine_info)

In [953]:
#전체 묶음 개수 확인
print(f"총 날짜 묶음 수: {len(sampled_orders)}")

# 전체 날짜별 주문 수 확인
order_counts_by_date = order_info_nl_eng.groupby('time').size().reset_index(name='order_count')
display(order_counts_by_date.head(10))  # 상위 10개만 보기


총 날짜 묶음 수: 365


,time,order_count
0,date001,55
1,date002,55
2,date003,55
3,date004,55
4,date005,55
5,date006,55
6,date007,55
7,date008,55
8,date009,55
9,date010,55


### 병렬 처리
---


In [930]:
# 사용할 고정 하이퍼파라미터 
ga_fixed_hyperparams = {
    'n_iter': 500,
    'n_pop': 20,
    'r_cross': 0.4,
    'r_mut': 0.1,
    'max_machine_work_time': 600,
    'overproduction_penalty_factor': 10000000,
    'gene_swap_prob': 0.5 # 우리가 교차 연산자에 추가했던 확률
}
print(f"사용할 고정 하이퍼파라미터: {ga_fixed_hyperparams}")

if 'dataset_for_ga' in globals() and dataset_for_ga: 

    # 병렬 처리 실행 및 시간 측정
    start_time = time.time()
    final_scores, final_gt_list, final_logs = \
        ga_runner.generate_gt_in_parallel(
            dataset_for_ga, 
            fixed_hyperparams,
            output_pickle_path=GT_N5_PKL_FILE,
            verbose=VERBOSE_VALUE
        )
    
    end_time = time.time()

    print(f"\n--- 병렬 처리 실행 요약 ---")
    print(f"총 {len(final_scores)}개의 샘플 처리 완료.")
    print(f"소요 시간: {end_time - start_time:.2f} 초")

    # 처리 결과 확인
    print(f"\n첫 번째 샘플의 Best Score: {final_scores[0] if final_scores else 'N/A'}")
    print("첫 번째 샘플의 GT DataFrame (상위 5개):")
    if final_gt_list and not final_gt_list[0].empty:
        print(final_gt_list[0].head())
    else:
        print("(데이터 없음)")

else:
    print("dataset_for_ga가 준비되지 않아 실행할 수 없습니다.")


사용할 고정 하이퍼파라미터: {'n_iter': 500, 'n_pop': 20, 'r_cross': 0.4, 'r_mut': 0.1, 'max_machine_work_time': 600, 'overproduction_penalty_factor': 10000000, 'gene_swap_prob': 0.5}
병렬 처리에 사용할 CPU 코어(프로세스) 수: 22

총 365개의 샘플에 대해 병렬 처리를 시작합니다...


100%|████████████████████████████████████████████████████████████████████████████| 365/365 [02:33<00:00,  2.38it/s]

모든 샘플에 대한 병렬 처리 완료!

최종 GT 리스트 저장 완료: gt_n5_augmented_20250613_divide_date.pkl

--- 병렬 처리 실행 요약 ---
총 365개의 샘플 처리 완료.
소요 시간: 160.26 초

첫 번째 샘플의 Best Score: 44572596
첫 번째 샘플의 GT DataFrame (상위 5개):
       item    machine     time  qty
0   item127  machine04  date001    1
1   item127  machine31  date001    1
2   item127  machine34  date001    1
3  item1289  machine09  date001    4
4  item1289  machine10  date001    3


In [931]:
with open(GT_N5_PKL_FILE, 'rb') as f:
    refactored_gt_ls = pickle.load(f)

refactored_gt_ls[10]

,item,machine,time,qty
0,item1001,machine32,date011,2
1,item1045,machine04,date011,5
2,item1045,machine05,date011,1
3,item1045,machine08,date011,7
4,item1045,machine09,date011,2
...,...,...,...,...
172,item80,machine22,date011,1
173,item846,machine06,date011,7
174,item846,machine21,date011,4
175,item846,machine23,date011,8


In [932]:
# GT 결과값 데이테프레임화
# for idx, gt in enumerate(refactored_gt_ls):
#     gt[['item', 'machine', 'time']] = pd.DataFrame(gt['(item, machine, time)'].tolist(), index=gt.index)
#     gt = gt.drop(columns='(item, machine, time)')
#     gt = gt[['item', 'machine', 'time', 'qty']].sort_values(['time', 'item', 'machine', 'qty'])
#     refactored_gt_ls[idx] = gt

refactored_gt_ls[0]

,item,machine,time,qty
0,item127,machine04,date001,1
1,item127,machine31,date001,1
2,item127,machine34,date001,1
3,item1289,machine09,date001,4
4,item1289,machine10,date001,3
...,...,...,...,...
144,item733,machine35,date001,9
145,item769,machine21,date001,2
146,item829,machine21,date001,13
147,item954,machine01,date001,5


In [933]:
# order 정보 묶기
def combine_order_nl(data):
    combined_data = '\n'.join(f'• {text}' for text in data['nl_order'])

    return combined_data

combine_order_nl(dataset_for_nl[0])

'• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan'

In [934]:
order_nl_for_dataset = [combine_order_nl(data) for data in dataset_for_nl]
order_nl_for_dataset[0]

'• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• nan'

In [935]:
dataset_for_order[0]

,time,item,cost,urgent,qty
0,date001,item127,9195,0,4
1,date001,item1289,33640,1,13
2,date001,item1316,6498,0,1
3,date001,item1508,179208,0,16
4,date001,item1830,52115,0,22
5,date001,item1976,104092,0,42
6,date001,item2179,6498,0,1
7,date001,item2245,179208,0,48
8,date001,item2308,6498,0,6
9,date001,item2321,104092,0,102


In [936]:
dataset_for_ga[0]

,time,item,cost,urgent,qty,machine,capacity
0,date001,item127,9195,0,4,machine04,4.25
1,date001,item127,9195,0,4,machine09,3.90
2,date001,item127,9195,0,4,machine24,5.00
3,date001,item127,9195,0,4,machine31,4.50
4,date001,item127,9195,0,4,machine34,4.41
...,...,...,...,...,...,...,...
172,date001,item733,18303,0,118,machine35,5.50
173,date001,item769,6498,0,2,machine21,1.00
174,date001,item829,179208,1,13,machine21,13.50
175,date001,item954,179208,1,138,machine01,34.33


In [937]:
# order machine join 정보 묶기
def combine_order_machine_join(data):
    combined_data = list()
    for _, row in data.iterrows():
        combined_data.append(f"• {row['time']} {row['item']} {row['cost']} {row['urgent']} {row['qty']} {row['machine']} {row['capacity']}")

    combined_data = "\n".join(combined_data)
        
    return combined_data

combine_order_machine_join(dataset_for_ga[0])

'• date001 item127 9195 0 4 machine04 4.25\n• date001 item127 9195 0 4 machine09 3.9\n• date001 item127 9195 0 4 machine24 5.0\n• date001 item127 9195 0 4 machine31 4.5\n• date001 item127 9195 0 4 machine34 4.41\n• date001 item1289 33640 1 13 machine09 5.67\n• date001 item1289 33640 1 13 machine10 6.17\n• date001 item1289 33640 1 13 machine28 2.0\n• date001 item1316 6498 0 1 machine28 2.0\n• date001 item1316 6498 0 1 machine29 1.15\n• date001 item1508 179208 0 16 machine05 33.07\n• date001 item1830 52115 0 22 machine23 6.0\n• date001 item1830 52115 0 22 machine28 5.0\n• date001 item1976 104092 0 42 machine03 44.5\n• date001 item1976 104092 0 42 machine05 37.0\n• date001 item1976 104092 0 42 machine06 52.0\n• date001 item1976 104092 0 42 machine19 41.5\n• date001 item1976 104092 0 42 machine20 38.05\n• date001 item2179 6498 0 1 machine28 1.88\n• date001 item2245 179208 0 48 machine03 11.5\n• date001 item2245 179208 0 48 machine06 37.5\n• date001 item2308 6498 0 6 machine08 3.17\n• date0

In [938]:
order_machine_join = [combine_order_machine_join(data) for data in dataset_for_ga]
print(order_machine_join[0])

• date001 item127 9195 0 4 machine04 4.25
• date001 item127 9195 0 4 machine09 3.9
• date001 item127 9195 0 4 machine24 5.0
• date001 item127 9195 0 4 machine31 4.5
• date001 item127 9195 0 4 machine34 4.41
• date001 item1289 33640 1 13 machine09 5.67
• date001 item1289 33640 1 13 machine10 6.17
• date001 item1289 33640 1 13 machine28 2.0
• date001 item1316 6498 0 1 machine28 2.0
• date001 item1316 6498 0 1 machine29 1.15
• date001 item1508 179208 0 16 machine05 33.07
• date001 item1830 52115 0 22 machine23 6.0
• date001 item1830 52115 0 22 machine28 5.0
• date001 item1976 104092 0 42 machine03 44.5
• date001 item1976 104092 0 42 machine05 37.0
• date001 item1976 104092 0 42 machine06 52.0
• date001 item1976 104092 0 42 machine19 41.5
• date001 item1976 104092 0 42 machine20 38.05
• date001 item2179 6498 0 1 machine28 1.88
• date001 item2245 179208 0 48 machine03 11.5
• date001 item2245 179208 0 48 machine06 37.5
• date001 item2308 6498 0 6 machine08 3.17
• date001 item2321 104092 0 10

In [939]:
def combine_order(data):
    combined_data = list()
    for _, row in data.iterrows():
        combined_data.append(f"• {row['time']} {row['item']} {row['cost']} {row['urgent']} {row['qty']}")

    combined_data = "\n".join(combined_data)
        
    return combined_data

print(combine_order(dataset_for_order[0]))

• date001 item127 9195 0 4
• date001 item1289 33640 1 13
• date001 item1316 6498 0 1
• date001 item1508 179208 0 16
• date001 item1830 52115 0 22
• date001 item1976 104092 0 42
• date001 item2179 6498 0 1
• date001 item2245 179208 0 48
• date001 item2308 6498 0 6
• date001 item2321 104092 0 102
• date001 item2512 86821 0 4
• date001 item2681 20470 0 50
• date001 item2699 106952 1 12
• date001 item2844 15218 1 5
• date001 item2934 8065 1 2
• date001 item3205 77242 1 13
• date001 item3238 81204 0 16
• date001 item330 6332 0 2
• date001 item3370 44803 0 42
• date001 item3542 101291 0 4
• date001 item365 8416 0 13
• date001 item3768 99504 0 7
• date001 item392 39325 0 78
• date001 item3954 179208 0 22
• date001 item4038 104092 0 75
• date001 item4076 6498 0 1
• date001 item4116 14698 0 21
• date001 item4187 6498 0 1
• date001 item4403 52115 0 2
• date001 item4609 6498 0 4
• date001 item4642 86821 0 3
• date001 item4828 109083 0 46
• date001 item4842 27256 0 14
• date001 item4852 35875 0 94

In [940]:
order_for_dataset = [combine_order(data) for data in dataset_for_ga]
print(f"원본 리스트 길이: {len(order_for_dataset)}")
order_for_dataset[0]


원본 리스트 길이: 365


'• date001 item127 9195 0 4\n• date001 item127 9195 0 4\n• date001 item127 9195 0 4\n• date001 item127 9195 0 4\n• date001 item127 9195 0 4\n• date001 item1289 33640 1 13\n• date001 item1289 33640 1 13\n• date001 item1289 33640 1 13\n• date001 item1316 6498 0 1\n• date001 item1316 6498 0 1\n• date001 item1508 179208 0 16\n• date001 item1830 52115 0 22\n• date001 item1830 52115 0 22\n• date001 item1976 104092 0 42\n• date001 item1976 104092 0 42\n• date001 item1976 104092 0 42\n• date001 item1976 104092 0 42\n• date001 item1976 104092 0 42\n• date001 item2179 6498 0 1\n• date001 item2245 179208 0 48\n• date001 item2245 179208 0 48\n• date001 item2308 6498 0 6\n• date001 item2321 104092 0 102\n• date001 item2321 104092 0 102\n• date001 item2321 104092 0 102\n• date001 item2321 104092 0 102\n• date001 item2321 104092 0 102\n• date001 item2321 104092 0 102\n• date001 item2512 86821 0 4\n• date001 item2681 20470 0 50\n• date001 item2681 20470 0 50\n• date001 item2681 20470 0 50\n• date001 i

In [941]:
def combine_machine(data):
    combined_data = list()
    for _, row in data.iterrows():
        combined_data.append(f"• {row['item']} {row['machine']} {row['capacity']}")

    combined_data = list(set(combined_data))
    combined_data = "\n".join(combined_data)
        
    return combined_data

print(combine_machine(dataset_for_ga[0]))

• item1316 machine28 2.0
• item5326 machine22 2.55
• item733 machine31 1.72
• item2699 machine01 10.0
• item509 machine30 0.77
• item5856 machine09 2.31
• item4828 machine28 8.0
• item6320 machine10 28.27
• item5979 machine32 2.5
• item3954 machine29 11.22
• item6320 machine23 29.6
• item2681 machine21 2.3
• item2321 machine23 40.0
• item3370 machine14 13.0
• item2844 machine32 6.0
• item2844 machine33 5.17
• item733 machine30 4.46
• item4852 machine29 6.3
• item127 machine24 5.0
• item4038 machine19 20.0
• item4609 machine10 2.0
• item6320 machine14 26.0
• item5399 machine01 30.0
• item2699 machine04 12.0
• item4076 machine23 3.0
• item546 machine12 5.79
• item733 machine14 7.22
• item365 machine09 2.64
• item365 machine22 2.29
• item4852 machine08 7.71
• item5235 machine08 1.38
• item5235 machine14 4.89
• item4852 machine21 4.83
• item5856 machine21 2.4
• item4852 machine30 7.9
• item392 machine08 14.67
• item5856 machine23 1.87
• item954 machine05 35.34
• item733 machine32 6.9
• ite

In [942]:
machine_for_dataset = [combine_machine(data) for data in dataset_for_ga]
machine_for_dataset[0]

'• item1316 machine28 2.0\n• item5326 machine22 2.55\n• item733 machine31 1.72\n• item2699 machine01 10.0\n• item509 machine30 0.77\n• item5856 machine09 2.31\n• item4828 machine28 8.0\n• item6320 machine10 28.27\n• item5979 machine32 2.5\n• item3954 machine29 11.22\n• item6320 machine23 29.6\n• item2681 machine21 2.3\n• item2321 machine23 40.0\n• item3370 machine14 13.0\n• item2844 machine32 6.0\n• item2844 machine33 5.17\n• item733 machine30 4.46\n• item4852 machine29 6.3\n• item127 machine24 5.0\n• item4038 machine19 20.0\n• item4609 machine10 2.0\n• item6320 machine14 26.0\n• item5399 machine01 30.0\n• item2699 machine04 12.0\n• item4076 machine23 3.0\n• item546 machine12 5.79\n• item733 machine14 7.22\n• item365 machine09 2.64\n• item365 machine22 2.29\n• item4852 machine08 7.71\n• item5235 machine08 1.38\n• item5235 machine14 4.89\n• item4852 machine21 4.83\n• item5856 machine21 2.4\n• item4852 machine30 7.9\n• item392 machine08 14.67\n• item5856 machine23 1.87\n• item954 machine

In [943]:
# gt 정보 텍스트화
def combine_gt(data):
    combined_data = list()
    for _, row in data.iterrows():
        combined_data.append(f"• {row['item']} {row['machine']} {row['time']} {row['qty']}")

    combined_data = "\n".join(combined_data)
        
    return combined_data

print(combine_gt(refactored_gt_ls[0]))

• item127 machine04 date001 1
• item127 machine31 date001 1
• item127 machine34 date001 1
• item1289 machine09 date001 4
• item1289 machine10 date001 3
• item1289 machine28 date001 6
• item1316 machine28 date001 1
• item1508 machine05 date001 1
• item1830 machine23 date001 1
• item1830 machine28 date001 1
• item1976 machine03 date001 2
• item1976 machine05 date001 2
• item1976 machine06 date001 1
• item1976 machine19 date001 2
• item1976 machine20 date001 1
• item2179 machine28 date001 1
• item2245 machine03 date001 14
• item2245 machine06 date001 5
• item2308 machine08 date001 6
• item2321 machine20 date001 5
• item2321 machine21 date001 9
• item2321 machine23 date001 16
• item2321 machine28 date001 9
• item2321 machine32 date001 22
• item2321 machine33 date001 5
• item2512 machine08 date001 2
• item2681 machine08 date001 6
• item2681 machine09 date001 7
• item2681 machine10 date001 10
• item2681 machine21 date001 4
• item2681 machine23 date001 5
• item2681 machine28 date001 5
• item2

In [944]:
refactored_gt_for_dataset = [combine_gt(data) for data in refactored_gt_ls]
refactored_gt_for_dataset[0]

'• item127 machine04 date001 1\n• item127 machine31 date001 1\n• item127 machine34 date001 1\n• item1289 machine09 date001 4\n• item1289 machine10 date001 3\n• item1289 machine28 date001 6\n• item1316 machine28 date001 1\n• item1508 machine05 date001 1\n• item1830 machine23 date001 1\n• item1830 machine28 date001 1\n• item1976 machine03 date001 2\n• item1976 machine05 date001 2\n• item1976 machine06 date001 1\n• item1976 machine19 date001 2\n• item1976 machine20 date001 1\n• item2179 machine28 date001 1\n• item2245 machine03 date001 14\n• item2245 machine06 date001 5\n• item2308 machine08 date001 6\n• item2321 machine20 date001 5\n• item2321 machine21 date001 9\n• item2321 machine23 date001 16\n• item2321 machine28 date001 9\n• item2321 machine32 date001 22\n• item2321 machine33 date001 5\n• item2512 machine08 date001 2\n• item2681 machine08 date001 6\n• item2681 machine09 date001 7\n• item2681 machine10 date001 10\n• item2681 machine21 date001 4\n• item2681 machine23 date001 5\n• item

In [945]:
refactored_dataset = pd.DataFrame(columns=['order', 'nl', 'machine', 'gt', 'objective'])
refactored_dataset

,order,nl,machine,gt,objective


In [946]:
print(f"final_scores 길이: {len(final_scores)}")

final_scores 길이: 365


In [947]:
refactored_dataset['order'] = order_for_dataset
refactored_dataset['nl'] = order_nl_for_dataset
refactored_dataset['machine'] = machine_for_dataset
refactored_dataset['gt'] = refactored_gt_for_dataset
refactored_dataset['objective'] = final_scores
refactored_dataset.head()

,order,nl,machine,gt,objective
0,• date001 item127 9195 0 4\n• date001 item127 ...,• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• na...,• item1316 machine28 2.0\n• item5326 machine22...,• item127 machine04 date001 1\n• item127 machi...,44572596
1,• date002 item1051 71585 0 138\n• date002 item...,• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• na...,• item4789 machine23 3.77\n• item3444 machine2...,• item1051 machine04 date002 9\n• item1051 mac...,11827728
2,• date003 item1130 109155 0 125\n• date003 ite...,"• Please check this, item2684 2 is until June ...",• item1233 machine09 2.66\n• item79 machine35 ...,• item1130 machine01 date003 21\n• item1130 ma...,0
3,• date004 item1124 179208 0 82\n• date004 item...,• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• na...,• item3471 machine08 5.32\n• item3583 machine2...,• item1124 machine04 date004 2\n• item1124 mac...,0
4,• date005 item1111 39310 0 66\n• date005 item1...,"• item4373, right? Doing 5? You can do it unti...",• item1221 machine28 2.55\n• item6610 machine2...,• item1111 machine10 date005 35\n• item1111 ma...,0


In [948]:
refactored_dataset['nl_machine'] = (
    "Order information\n" + refactored_dataset['nl'] + "\n\n" +
    "Machine information\n" + refactored_dataset['machine']
)

refactored_dataset['nl_machine_gt'] = (
    "Order information\n" + refactored_dataset['nl'] + "\n\n" +
    "Machine information\n" + refactored_dataset['machine'] + "\n\n" +
    "Schedule result\n" + refactored_dataset['gt']
)

refactored_dataset['order_machine'] = (
    "Order information\n" + refactored_dataset['order'] + "\n\n" +
    "Machine information\n" + refactored_dataset['machine']
)

refactored_dataset['order_machine_gt'] = (
    "Order information\n" + refactored_dataset['order'] + "\n\n" +
    "Machine information\n" + refactored_dataset['machine'] + "\n\n" +
    "Schedule result\n" + refactored_dataset['gt']
)

refactored_dataset['order_machine_join'] = order_machine_join

refactored_dataset['order_machine_join_gt'] = (
    "Order and machine information\n" + refactored_dataset['order_machine_join'] + "\n\n" +
    "Schedule result\n" + refactored_dataset['gt']
)

refactored_dataset.head()

,order,nl,machine,gt,objective,nl_machine,nl_machine_gt,order_machine,order_machine_gt,order_machine_join,order_machine_join_gt
0,• date001 item127 9195 0 4\n• date001 item127 ...,• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• na...,• item1316 machine28 2.0\n• item5326 machine22...,• item127 machine04 date001 1\n• item127 machi...,44572596,Order information\n• nan\n• nan\n• nan\n• nan\...,Order information\n• nan\n• nan\n• nan\n• nan\...,Order information\n• date001 item127 9195 0 4\...,Order information\n• date001 item127 9195 0 4\...,• date001 item127 9195 0 4 machine04 4.25\n• d...,Order and machine information\n• date001 item1...
1,• date002 item1051 71585 0 138\n• date002 item...,• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• na...,• item4789 machine23 3.77\n• item3444 machine2...,• item1051 machine04 date002 9\n• item1051 mac...,11827728,Order information\n• nan\n• nan\n• nan\n• nan\...,Order information\n• nan\n• nan\n• nan\n• nan\...,Order information\n• date002 item1051 71585 0 ...,Order information\n• date002 item1051 71585 0 ...,• date002 item1051 71585 0 138 machine04 5.0\n...,Order and machine information\n• date002 item1...
2,• date003 item1130 109155 0 125\n• date003 ite...,"• Please check this, item2684 2 is until June ...",• item1233 machine09 2.66\n• item79 machine35 ...,• item1130 machine01 date003 21\n• item1130 ma...,0,"Order information\n• Please check this, item26...","Order information\n• Please check this, item26...",Order information\n• date003 item1130 109155 0...,Order information\n• date003 item1130 109155 0...,• date003 item1130 109155 0 125 machine01 15.0...,Order and machine information\n• date003 item1...
3,• date004 item1124 179208 0 82\n• date004 item...,• nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• na...,• item3471 machine08 5.32\n• item3583 machine2...,• item1124 machine04 date004 2\n• item1124 mac...,0,Order information\n• nan\n• nan\n• nan\n• nan\...,Order information\n• nan\n• nan\n• nan\n• nan\...,Order information\n• date004 item1124 179208 0...,Order information\n• date004 item1124 179208 0...,• date004 item1124 179208 0 82 machine04 18.17...,Order and machine information\n• date004 item1...
4,• date005 item1111 39310 0 66\n• date005 item1...,"• item4373, right? Doing 5? You can do it unti...",• item1221 machine28 2.55\n• item6610 machine2...,• item1111 machine10 date005 35\n• item1111 ma...,0,"Order information\n• item4373, right? Doing 5?...","Order information\n• item4373, right? Doing 5?...",Order information\n• date005 item1111 39310 0 ...,Order information\n• date005 item1111 39310 0 ...,• date005 item1111 39310 0 66 machine10 6.97\n...,Order and machine information\n• date005 item1...


In [949]:
print(refactored_dataset.iloc[0, -3])

Order information
• date001 item127 9195 0 4
• date001 item127 9195 0 4
• date001 item127 9195 0 4
• date001 item127 9195 0 4
• date001 item127 9195 0 4
• date001 item1289 33640 1 13
• date001 item1289 33640 1 13
• date001 item1289 33640 1 13
• date001 item1316 6498 0 1
• date001 item1316 6498 0 1
• date001 item1508 179208 0 16
• date001 item1830 52115 0 22
• date001 item1830 52115 0 22
• date001 item1976 104092 0 42
• date001 item1976 104092 0 42
• date001 item1976 104092 0 42
• date001 item1976 104092 0 42
• date001 item1976 104092 0 42
• date001 item2179 6498 0 1
• date001 item2245 179208 0 48
• date001 item2245 179208 0 48
• date001 item2308 6498 0 6
• date001 item2321 104092 0 102
• date001 item2321 104092 0 102
• date001 item2321 104092 0 102
• date001 item2321 104092 0 102
• date001 item2321 104092 0 102
• date001 item2321 104092 0 102
• date001 item2512 86821 0 4
• date001 item2681 20470 0 50
• date001 item2681 20470 0 50
• date001 item2681 20470 0 50
• date001 item2681 20470 0

In [950]:
refactored_dataset.to_csv(REFACTORY_GT_N5_FILE)

**규범님이 order 컬럼에 동일 데이터가 중복 되어 토근 낭비 된다고하여 중복 제거**

In [951]:
def remove_duplicate_lines(multiline_string):
    """
    하나의 긴 문자열(여러 줄로 구성)을 입력받아,
    내부의 중복된 라인을 제거하고 순서를 유지한 새로운 문자열을 반환합니다.
    """
    if not isinstance(multiline_string, str):
        return multiline_string
    
    lines = multiline_string.split('\n')
    # 혹시 모를 빈 라인 항목은 제거
    non_empty_lines = [line for line in lines if line.strip()]
    unique_lines = list(dict.fromkeys(non_empty_lines))
    return '\n'.join(unique_lines)


try:
    df = pd.read_csv(REFACTORY_GT_N5_FILE)
    print(f"'{REFACTORY_GT_N5_FILE}' 파일 로드 성공. 총 {len(df)}개 행.")

    df['order_unique'] = df['order'].apply(remove_duplicate_lines)

    # 기존 'order' 컬럼을 삭제
    df = df.drop(columns=['order'])
    
    # 새로 만든 'order_unique' 컬럼의 이름을 'order'로 변경
    df = df.rename(columns={'order_unique': 'order'})
    
    # 컬럼 순서를 사용자가 원하는 대로 재정렬합니다.
    print("\n'order' 컬럼을 두 번째 위치로 재정렬합니다...")
    
    all_columns = df.columns.tolist()
    
    # 'order' 컬럼을 리스트에서 일단 제거합니다.
    all_columns.remove('order')
    
    # 리스트의 두 번째 위치(인덱스 1)에 'order' 컬럼 이름을 다시 삽입합니다.
    all_columns.insert(1, 'order')
    
    # 이 새로운 순서 리스트를 사용해서 DataFrame의 컬럼 순서를 재정의합니다.
    df = df[all_columns]

    # 7. 수정된 DataFrame을 새로운 CSV 파일로 저장
    #    (원본 파일을 덮어쓰지 않도록 새로운 파일 이름을 사용)
    output_filename = REFACTORY_GT_N5_FILE.replace('.csv', '_deduplicated.csv')
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    
    print(f"\n중복 라인이 제거된 데이터가 '{output_filename}' 파일로 저장되었습니다.")
    print("\n--- 저장된 DataFrame (상위 5개) ---")
    print(df.head())

except FileNotFoundError:
    print(f"오류: '{REFACTORY_GT_N5_FILE}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")

'./augmented_dataset_n5_20250613_divide_date.csv' 파일 로드 성공. 총 365개 행.

'order' 컬럼을 두 번째 위치로 재정렬합니다...

중복 라인이 제거된 데이터가 './augmented_dataset_n5_20250613_divide_date_deduplicated.csv' 파일로 저장되었습니다.

--- 저장된 DataFrame (상위 5개) ---
   Unnamed: 0                                              order  \
0           0  • date001 item127 9195 0 4\n• date001 item1289...   
1           1  • date002 item1051 71585 0 138\n• date002 item...   
2           2  • date003 item1130 109155 0 125\n• date003 ite...   
3           3  • date004 item1124 179208 0 82\n• date004 item...   
4           4  • date005 item1111 39310 0 66\n• date005 item1...   

                                                  nl  \
0  • nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• na...   
1  • nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• na...   
2  • Please check this, item2684 2 is until June ...   
3  • nan\n• nan\n• nan\n• nan\n• nan\n• nan\n• na...   
4  • item4373, right? Doing 5? You can do it unti...   

                            